In [17]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import pickle
import random

#from preprocessing_1_initial_scrape import *

In [2]:
# Retrieve dblp_df with pickle
with open('dblp_12k_processed_df.pkl', 'rb') as f:
    dblp_df = pickle.load(f)

In [3]:
def filter_unique_authors(dblp_df):
    # Iterate to fill up list of other authors
    other_authors_list = []
    for i, row in dblp_df.iterrows():
        for each in row['Other Authors']:
            other_authors_list.append(each)

    # Convert to retrieve only unique author entries
    other_authors_set = set(other_authors_list)
    unique_authors_list = list(other_authors_set)

    # Iterate to retrieve unique authors with distinguishable names
    unique_authors_numberless_list = []
    for each in unique_authors_list:
        if re.search(r'\d', each) == None:    
            unique_authors_numberless_list.append(each)

8999

# Remove all SCSE faculty members

In [4]:
# Retrieve unique faculty names


In [5]:
# iterate over this list to check and remove scse people
def remove_scse(main_list):

    faculty_list = dblp_df['Faculty'].tolist()
    unique_faculty_list = list(set(faculty_list))
    
    for each in unique_faculty_list:

        mod_fac_name = each.replace("-", " ")
        mod_fac_name_list = mod_fac_name.split(" ")
        #print(mod_fac_name_list)
        best_i = 0
        best_count = 0

        for i in range(len(main_list)):
            #check matchability
            author = main_list[i]
            count = 0
            for w in mod_fac_name_list:
                if w in author:
                    count += 1
            if count > best_count:
                best_count = count
                best_i = i
        main_list.remove(main_list[best_i])
    return main_list
        
unique_authors_numberless_scseless_list = remove_scse(unique_authors_numberless_list)
len(unique_authors_numberless_scseless_list)

8914

In [6]:
def retrieve_thousand_faculty(dblp_df):
    ssb_df = dblp_df[dblp_df['Faculty'] == 'Sourav S Bhowmick']

    # Iterate to fill up list of other authors
    ssb_other_authors_list = []
    for i, row in ssb_df.iterrows():
        for each in row['Other Authors']:
            ssb_other_authors_list.append(each)

    # Convert to retrieve only unique author entries
    ssb_unique_authors_list = list(set(ssb_other_authors_list))

    # Iterate to retrieve unique authors with distinguishable names
    ssb_unique_authors_numberless_list = []
    for each in ssb_unique_authors_list:
        if re.search(r'\d', each) == None:    
            ssb_unique_authors_numberless_list.append(each)

    # Cross-compare and retrieve only non-scse sourav pals
    sourav_pal_list = []
    for each in unique_authors_numberless_scseless_list:
        for ssb_each in ssb_unique_authors_numberless_list:
            if ssb_each == each:
                sourav_pal_list.append(ssb_each)
                
    # Retrieve non-sourav pal authors from main list
    remainder_authors_list = list(set(unique_authors_numberless_scseless_list).symmetric_difference(set(sourav_pal_list)))
    
    # randomly chosen remainder people
    random_chosen_list = random.sample(remainder_authors_list, 1000-len(sourav_pal_list))
    
    return sourav_pal_list + random_chosen_list

In [24]:
# Store dblp_df with pickle
with open('1000_faculty.pkl', 'wb') as f:
    pickle.dump(thousand_apostles_list, f)